In [72]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import *
from keras import initializers
from tensorflow.keras import regularizers
import keras
from sklearn.preprocessing import *

In [34]:
#Clean the cosmetic data
cos_df = pd.read_csv('data/Cosmetic_products_sales.csv')

cos_df = cos_df.drop(columns=['Unit',"Pack Unit Id",'Master Category','Size','Rank','Date','Amount to Customer']).dropna()

default = [4,2017]
current = default.copy()

missing_times = []
last = ''
past = 0

counter = 0
# Iterate through each row in the dataframe
for index, row in cos_df.iterrows():
    curr = row['Site Id']
    month = row['Month']
    year = row['Year']

    counter += 1
    if curr != last:
        current = default.copy()
        past = row["Net Sales calculated"]
        last = curr

    while month != current[0] or year != current[1]:
        dup = row.copy()
        dup['Month'] = current[0]
        dup['Year'] = current[1]
        
        missing_times.append(dup.tolist())
        current[0] += 1
        if current[0] > 12:
            current[0] = 1
            current[1] += 1
    current[0] += 1
    if current[0] > 12:
        current[0] = 1
        current[1] += 1
    # cos_df.loc[index, "Net Sales calculated"] -= past

missing_df = pd.DataFrame(missing_times, columns=cos_df.columns)
cos_df = pd.concat([cos_df, missing_df], ignore_index=True)

cos_df['Date'] = cos_df['Year'].astype(str) + '-' + cos_df['Month'].astype(str)

# Convert the combined column to datetime
cos_df['Date'] = pd.to_datetime(cos_df['Date'])

# Drop the original year and month columns if needed
cos_df = cos_df.drop(columns=['Year', 'Month'])

string_encoded = ['ParentSKU', 'Site Id','State','Zone']
# normalize = ["Net Sales calculated","Cash Discount","Amount to Customer","Qty","Price","MRP","Pack Size"]
cos_df_sales = cos_df["Net Sales calculated"]

for i in cos_df.columns:
    if i in string_encoded:
        cos_df[i] = pd.factorize(cos_df[i])[0]
        
cos_df = cos_df.sort_values(by=['ParentSKU','Site Id','Date']).reset_index(drop=True)
cos_df

,ParentSKU,Site Id,Category Name ID,Qty,Price,Net Sales calculated,Cash Discount,MRP,Pack Size,State,Zone,Date
0,0,0,0,1621.0,54,82557,20,80.0,30.0,0,0,2017-04-01
1,0,0,0,651.0,51,31473,21,80.0,30.0,0,0,2017-05-01
2,0,0,0,457.0,46,19580,12,70.0,30.0,0,0,2017-06-01
3,0,0,0,1985.0,41,78144,0,70.0,30.0,0,0,2017-07-01
4,0,0,0,6.0,38,228,0,70.0,30.0,0,0,2017-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...
42206,51,24,1,1179.0,40,44215,0,60.0,120.0,0,3,2019-10-01
42207,51,24,1,81.0,40,2947,0,60.0,120.0,0,3,2019-11-01
42208,51,24,1,81.0,40,2947,0,60.0,120.0,0,3,2019-12-01
42209,51,24,1,580.0,40,21125,0,60.0,120.0,0,3,2020-01-01


In [35]:
cos_df = cos_df.drop(columns=['Cash Discount','MRP','Pack Size','State','Zone','Date','Category Name ID']).dropna()
# cos_df
#important define current row
curr_row = cos_df.columns.get_loc('Net Sales calculated') 
curr_row

4

In [36]:
scalar = StandardScaler()
scalar.fit(cos_df)
cos_data = scalar.transform(cos_df)
cos_data
a = scalar.inverse_transform(cos_data)

In [37]:
def timeseries_cos_data(cos_data ,cos_answers,window_size=6,sample_size=33):
    cos_np = cos_data.astype(float) 
    x = []
    y = []
    for i in range(len(cos_np)//sample_size):
        for j in range(sample_size-window_size):
            row = [k for k in cos_np[i*sample_size+j:i*sample_size+j+window_size]]    
            x.append(row)
            y.append(cos_data[i*sample_size+j+window_size][curr_row].astype(float))
    return np.array(x), np.array(y)
cos_x, cos_y = timeseries_cos_data(cos_data,cos_df_sales)
cos_y = cos_y.reshape(cos_y.shape[0],1)

In [38]:
limit = 30000

cos_train_x, cos_train_y = cos_x[:limit],cos_y[:limit]
cos_test_x, cos_test_y = cos_x[limit:],cos_y[limit:]
cos_train_x.shape, cos_train_y.shape

((30000, 6, 5), (30000, 1))

In [74]:
model4 = Sequential()
model4.add(InputLayer((cos_train_x.shape[1], cos_train_x.shape[2])))  
model4.add(LSTM(128, activation='relu',kernel_regularizer=regularizers.l2(0.01), return_sequences=True)) 
model4.add(LSTM(64, activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))  
model4.add(LSTM(32, activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))  
model4.add(LSTM(16, activation='relu',kernel_regularizer=regularizers.l2(0.01),return_sequences=True))  
model4.add(LSTM(8, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model4.add(Dense(8, activation='relu'))
model4.add(Dense(cos_train_y.shape[1]))  # Output layer

model4.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_29 (LSTM)                  │ (None, 6, 128)         │        68,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_30 (LSTM)                  │ (None, 6, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_31 (LSTM)                  │ (None, 6, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_32 (LSTM)                  │ (None, 6, 16)          │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_33 (LSTM)                  │ (None, 8)              │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,449 (525.19 KB)

 Trainable params: 134,449 (525.19 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
model4 = keras.models.load_model("data/model4/my_model.keras")

In [70]:
from keras import callbacks
from keras.callbacks import EarlyStopping, LearningRateScheduler

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Define the learning rate schedule function
def lr_schedule(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.0001

# Define the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping and learning rate scheduling
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=32, callbacks=[early_stopping, lr_scheduler])


469/469 ━━━━━━━━━━━━━━━━━━━━ 236s 147ms/step - loss: 0.0916 - val_loss: 2.6966


In [54]:
cos_predictions = model4.predict(cos_test_x).flatten()

142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step


In [68]:
prediction_copies = np.repeat(cos_predictions, cos_test_x.shape[2], axis=0).reshape(cos_predictions.shape[0], cos_test_x.shape[2])
all = np.repeat(cos_train_y, cos_train_x.shape[2]).reshape(cos_train_y.shape[0], cos_train_x.shape[2])
all

array([[ 1.35828557,  1.35828557,  1.35828557,  1.35828557,  1.35828557],
       [ 1.13499841,  1.13499841,  1.13499841,  1.13499841,  1.13499841],
       [ 0.00240626,  0.00240626,  0.00240626,  0.00240626,  0.00240626],
       ...,
       [-0.3186318 , -0.3186318 , -0.3186318 , -0.3186318 , -0.3186318 ],
       [-0.19245302, -0.19245302, -0.19245302, -0.19245302, -0.19245302],
       [-0.16737385, -0.16737385, -0.16737385, -0.16737385, -0.16737385]])

In [69]:
y_pred_future = scalar.inverse_transform(prediction_copies)[:,curr_row]
all_stif = scalar.inverse_transform(all)[:,curr_row]
train_results = pd.DataFrame(data={'Train Predictions':y_pred_future, 'Actuals':all_stif})
train_results

,Train Predictions,Actuals
0,130180.617188,402769.0
1,257360.578125,350293.0
2,259622.218750,84116.0
3,154100.781250,132256.0
4,140682.593750,16717.0
...,...,...
4528,164029.859375,91630.0
4529,130069.398438,32070.0
4530,38211.664062,8667.0
4531,47623.625000,38321.0


In [71]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(all_stif, y_pred_future)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(all_stif, y_pred_future)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2) Score
r2 = r2_score(all_stif, y_pred_future)

# Print the results
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 151432351447.38922
Mean Absolute Error (MAE): 85944.44912329926
Root Mean Squared Error (RMSE): 389143.0989332706
R-squared (R2) Score: 0.48356645486016336
